In [1]:
import xgboost as xgb 
import pandas as pd 

from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder 
from sklearn.model_selection import train_test_split 
from sklearn.impute import SimpleImputer 

In [2]:
data = pd.read_csv('cirrhosis.csv')
necessary_cols = ['Status', 'Age', 'Sex', 'Cholesterol', 'Stage', 'Hepatomegaly', 'Bilirubin', 'Albumin', 'Platelets', 'Drug']

data = data[necessary_cols]
data 

,Status,Age,Sex,Cholesterol,Stage,Hepatomegaly,Bilirubin,Albumin,Platelets,Drug
0,D,21464,F,261.0,4.0,Y,14.5,2.60,190.0,D-penicillamine
1,C,20617,F,302.0,3.0,Y,1.1,4.14,221.0,D-penicillamine
2,D,25594,M,176.0,4.0,N,1.4,3.48,151.0,D-penicillamine
3,D,19994,F,244.0,4.0,Y,1.8,2.54,183.0,D-penicillamine
4,CL,13918,F,279.0,3.0,Y,3.4,3.53,136.0,Placebo
...,...,...,...,...,...,...,...,...,...,...
413,D,24472,F,NaN,3.0,NaN,1.2,2.96,174.0,NaN
414,C,14245,F,NaN,4.0,NaN,0.9,3.83,180.0,NaN
415,C,20819,F,NaN,3.0,NaN,1.6,3.42,143.0,NaN
416,C,21185,F,NaN,3.0,NaN,0.8,3.75,269.0,NaN


In [3]:
data.columns

Index(['Status', 'Age', 'Sex', 'Cholesterol', 'Stage', 'Hepatomegaly',
       'Bilirubin', 'Albumin', 'Platelets', 'Drug'],
      dtype='object')

In [4]:
data['Status'].value_counts()

Status
C     232
D     161
CL     25
Name: count, dtype: int64

In [5]:
data['Hepatomegaly'] = (data['Hepatomegaly'] == 'Y').astype(int)
data['Sex'] = (data['Sex'] == 'M').astype(int)


num_cols = ['Bilirubin', 'Albumin', 'Platelets', 'Cholesterol']
categorical_cols = ['Drug', 'Stage']
target_col = ['Status']

num_imputer = SimpleImputer(strategy='constant')
encoder = OrdinalEncoder()
health_data = data.copy()

health_data[target_col] = encoder.fit_transform(data[target_col])
health_data


,Status,Age,Sex,Cholesterol,Stage,Hepatomegaly,Bilirubin,Albumin,Platelets,Drug
0,2.0,21464,0,261.0,4.0,1,14.5,2.60,190.0,D-penicillamine
1,0.0,20617,0,302.0,3.0,1,1.1,4.14,221.0,D-penicillamine
2,2.0,25594,1,176.0,4.0,0,1.4,3.48,151.0,D-penicillamine
3,2.0,19994,0,244.0,4.0,1,1.8,2.54,183.0,D-penicillamine
4,1.0,13918,0,279.0,3.0,1,3.4,3.53,136.0,Placebo
...,...,...,...,...,...,...,...,...,...,...
413,2.0,24472,0,NaN,3.0,0,1.2,2.96,174.0,NaN
414,0.0,14245,0,NaN,4.0,0,0.9,3.83,180.0,NaN
415,0.0,20819,0,NaN,3.0,0,1.6,3.42,143.0,NaN
416,0.0,21185,0,NaN,3.0,0,0.8,3.75,269.0,NaN


In [11]:
cat_transform = Pipeline(steps=[
    ('imputer', (SimpleImputer(strategy='most_frequent'))),
    ('onehot', (OneHotEncoder(handle_unknown='ignore', sparse=False))),
    ('ordencode', (OrdinalEncoder()))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, num_cols),
        ('cat', cat_transform, categorical_cols),
        # ('enc', encoder, data['Status'])
    ]
)

In [12]:
x = health_data.drop(columns=['Status'])
y = health_data['Status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0) 


model = xgb.XGBClassifier(
                     objective='multi:softmax',
                     max_depth=5,
                     n_estimators=100,
                     random_state=42,
                     learning_rate=0.1,
                )

In [14]:
y_train.value_counts()

Status
0.0    185
2.0    129
1.0     20
Name: count, dtype: int64

In [13]:
my_pipeline = Pipeline(steps=[
                        ('preprocessor', preprocessor),
                        ('model', model)
                      ]) 

In [15]:
my_pipeline.fit(x_train, y_train)
survival_prediction = my_pipeline.predict(x_test) 
pred_accuracy = my_pipeline.score(x_test, y_test) 

pred_accuracy 

/home/kelechi/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.6666666666666666

In [16]:
metrics = classification_report(y_test,survival_prediction)

print(metrics)

              precision    recall  f1-score   support

         0.0       0.73      0.77      0.75        47
         1.0       0.00      0.00      0.00         5
         2.0       0.57      0.62      0.60        32

    accuracy                           0.67        84
   macro avg       0.44      0.46      0.45        84
weighted avg       0.63      0.67      0.65        84



/home/kelechi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kelechi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kelechi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
